# Work with database

Downloading the data from SQL database

The number of flights on each model of aircraft departing in September 2018.

In [ ]:
'''
SELECT
    DISTINCT aircrafts.model AS model,
    COUNT(flights.flight_id) AS flight_amount
FROM 
    aircrafts
INNER JOIN flights ON flights.aircraft_code = aircrafts.aircraft_code
WHERE
    flights.departure_time::date >= '2018-09-01' AND
    flights.departure_time::date < '2018-10-01'

GROUP BY aircrafts.model
'''

In [ ]:
'''
Airbus A319-100	607
Airbus A321-200	960
Boeing 737-300	630
Boeing 767-300	600
Boeing 777-300	300
Bombardier CRJ-200	4446
Cessna 208 Caravan	4557
Sukhoi SuperJet-100	4185
'''

The number of flights dispatched in September by the following models of aircraft:
* Boeing,
* Airbus,
* other ("other").

In [ ]:
'''
 SELECT
    CASE WHEN model LIKE '%Boeing%' THEN 'Boeing'
   WHEN model LIKE '%Airbus%' THEN 'Airbus'
   ELSE 
       'other'
   END AS model,
    COUNT(flights.flight_id) AS flight_amount
FROM 
    aircrafts
INNER JOIN flights ON flights.aircraft_code = aircrafts.aircraft_code
WHERE
    flights.departure_time::date >= '2018-09-01' AND
    flights.departure_time::date < '2018-10-01'

GROUP BY 
   CASE WHEN model LIKE '%Boeing%' THEN 'Boeing'
   WHEN model LIKE '%Airbus%' THEN 'Airbus'
   ELSE 
       'other'
   END
'''

In [ ]:
'''
Airbus 1567
Boeing 1530
other 13188
'''

Average number of arriving flights per day for each city for August 2018.

In [ ]:
'''
SELECT subq.city,
       avg(subq.num_flights) AS average_flights
FROM
    (SELECT airports.city AS city,
            count(flights.flight_id) AS num_flights,
            extract(DAY
                    FROM flights.arrival_time :: date) AS DAY
     FROM flights
     JOIN airports ON airports.airport_code = flights.arrival_airport
     WHERE (flights.arrival_time :: date < '2018-09-01'
            AND flights.arrival_time :: date >= '2018-08-01')
     GROUP BY airports.city,
              extract(DAY
                      FROM flights.arrival_time :: date)) AS subq
GROUP BY subq.city
ORDER BY average_flights
'''

In [ ]:
''' 
city average_flights
Москва 129.774
Санкт-Петербург 31.1613
Новосибирск 17.3226
Красноярск 11.5806
Екатеринбург 11.3226
'''

Festivals that took place from July 23 to September 30, 2018 in Moscow, and the number of the week in which they took place.

In [ ]:
'''
select 
    festivals.festival_name as festival_name,
    extract(week from festivals.festival_date) as festival_week
from 
    festivals
where
    festival_city = 'Москва' and
     festivals.festival_date::date >= '2018-07-23' and
     festivals.festival_date::date <= '2018-09-30'
'''

In [ ]:
'''
festival_name festival_week
Park Live 30
Пикник Афиши 31
Видфест 36
'''

For each week from July 23 to September 30, 2018, the tickets that were purchased for flights to Moscow were counted.  
A table is obtained:
* the number of tickets purchased for the week;
* mark whether there was a festival that week;
* name of the festival;
* number of the week.

In [ ]:
'''
SELECT 
	T.week_number,
	T.ticket_amount,
	T.festival_week,
	T.festival_name
FROM (
(
  SELECT EXTRACT(week from flights.arrival_time::date) as week_number,
          COUNT (ticket_flights.ticket_no) as ticket_amount
  FROM flights
  JOIN airports on airports.airport_code = flights.arrival_airport
  JOIN ticket_flights on ticket_flights.flight_id = flights.flight_id
  WHERE airports.city = 'Москва' and
        CAST(flights.arrival_time AS date) BETWEEN '2018-07-23' AND '2018-09-30'
  GROUP BY week_number
) t1
LEFT JOIN 
(SELECT 		
		festival_name,	
		EXTRACT (week FROM festivals.festival_date) AS festival_week
	FROM 
		festivals
	WHERE
		festival_city = 'Москва'
	  AND CAST(festivals.festival_date AS date) BETWEEN '2018-07-23' AND '2018-09-30'
) t2 
ON 
	t1.week_number = t2.festival_week
) AS T
'''

In [ ]:
'''
week_number ticket_amount festival_week festival_name
33 51378 nan 
31 51034 31 Пикник Афиши
38 51518 nan 
30 43568 30 Park Live
32 51675 na
34 51492 nan
3 51360 nan
37 51670 nan 
36 51386 36 Видфест
39 51623 nan 
'''